# Chapter 5: IDENTIFY (ID) – Asset & Risk Management

In Chapter 4, we established the governance structures that define our security strategy—policies that mandate protection, risk appetites that guide our decisions, and supply chain controls that secure our dependencies. However, strategy without intelligence is blind. Before we can protect our organization, before we can detect intrusions, or respond to incidents, we must answer a fundamental question: **What are we protecting?**

The **IDENTIFY** function of the NIST CSF 2.0 provides the organizational understanding to manage cybersecurity risk to systems, assets, data, and capabilities. This is not merely an inventory exercise; it is the continuous process of discovering what exists in our ever-changing environments, understanding the business context and criticality of those assets, modeling the threats they face, assessing their vulnerabilities, and quantifying the risks they present to our mission.

For developers, this chapter transforms abstract risk into concrete reality. We will learn to discover the APIs we have forgotten, classify the data we process, model the threats against our architectures before we write the first line of code, and prioritize vulnerabilities not by severity alone, but by the business impact of exploitation. We move from "I think our attack surface is small" to "We have 47 public-facing APIs, 12 contain PII, and 3 have critical vulnerabilities requiring patching within 24 hours."

---

## 5.1 Asset Discovery: Data, Systems, Applications, and APIs

Asset management is the foundation of all security operations. You cannot patch what you do not know exists, encrypt data you cannot find, or monitor logs from systems that are invisible to you.

### 5.1.1 The Asset Inventory Scope
Modern asset inventories extend far beyond physical servers. They include:

*   **Hardware:** Servers, laptops, mobile devices, IoT sensors, network appliances.
*   **Software:** Applications, microservices, serverless functions, databases, middleware.
*   **Data:** Structured (databases), unstructured (documents), ephemeral (logs, caches).
*   **Network Assets:** Subnets, cloud VPCs, VPN gateways, load balancers.
*   **Digital Identities:** Service accounts, API keys, certificates, user accounts.
*   **Cloud Resources:** S3 buckets, Lambda functions, Kubernetes clusters, IAM roles.

### 5.1.2 Shadow IT and the Ephemeral Challenge
**Shadow IT** refers to assets deployed without formal IT approval—often cloud services spun up by business units or developers testing new tools. By 2026, **ephemeral assets** (containers that exist for minutes, serverless functions invoked on-demand) create a dynamic inventory that traditional spreadsheets cannot capture.

**Automated Discovery with Cloud APIs:**
Continuous discovery is essential. This Python example uses the AWS Resource Groups Tagging API to discover untagged resources (a common indicator of Shadow IT):

```python
import boto3
import json
from datetime import datetime

class CloudAssetDiscovery:
    def __init__(self):
        self.tagging = boto3.client('resourcegroupstaggingapi')
        self.ec2 = boto3.client('ec2')
        self.inventory = []
    
    def discover_untagged_resources(self):
        """
        Identifies resources lacking mandatory security tags (Owner, Environment, DataClassification)
        Returns potential Shadow IT or unmanaged assets
        """
        paginator = self.tagging.get_paginator('get_resources')
        
        for page in paginator.paginate():
            for resource in page['ResourceTagMappingList']:
                tags = {tag['Key']: tag['Value'] for tag in resource.get('Tags', [])}
                
                mandatory_tags = ['Owner', 'Environment', 'DataClassification']
                missing_tags = [tag for tag in mandatory_tags if tag not in tags]
                
                if missing_tags:
                    self.inventory.append({
                        'arn': resource['ResourceARN'],
                        'resource_type': resource['ResourceType'],
                        'missing_tags': missing_tags,
                        'discovered_at': datetime.utcnow().isoformat(),
                        'risk_level': 'High' if 'DataClassification' in missing_tags else 'Medium'
                    })
        
        return self.inventory
    
    def discover_exposed_storage(self):
        """
        Finds S3 buckets that are publicly accessible (high risk data exposure)
        """
        s3 = boto3.client('s3')
        exposed_buckets = []
        
        buckets = s3.list_buckets()['Buckets']
        for bucket in buckets:
            try:
                # Check bucket policy
                policy = s3.get_bucket_policy_status(Bucket=bucket['Name'])
                if policy['PolicyStatus']['IsPublic']:
                    exposed_buckets.append({
                        'bucket': bucket['Name'],
                        'issue': 'PublicBucketPolicy',
                        'remediation': 'Review and restrict bucket policy'
                    })
            except s3.exceptions.ClientError as e:
                if 'NoSuchBucketPolicy' in str(e):
                    # Check ACLs if no policy exists
                    acl = s3.get_bucket_acl(Bucket=bucket['Name'])
                    for grant in acl['Grants']:
                        if grant['Grantee'].get('URI') == 'http://acs.amazonaws.com/groups/global/AllUsers':
                            exposed_buckets.append({
                                'bucket': bucket['Name'],
                                'issue': 'PublicACL',
                                'remediation': 'Remove public read/write ACLs'
                            })
        
        return exposed_buckets

# Usage
# scanner = CloudAssetDiscovery()
# shadow_assets = scanner.discover_untagged_resources()
# print(json.dumps(shadow_assets, indent=2))
```

### 5.1.3 API Discovery and Inventory
In modern microservices architectures, APIs are the primary attack surface. **API sprawl** occurs when services expose endpoints without centralized registration.

**OpenAPI Specification (OAS) as Inventory:**
Every API should have an OpenAPI (Swagger) specification that serves as its technical contract and security documentation.

```yaml
# Example OpenAPI 3.0 with security metadata
openapi: 3.0.3
info:
  title: Customer Data API
  version: 1.0.0
  description: |
    **Asset Classification:** Critical
    **Data Processed:** PII (Email, Phone)
    **Owner:** Platform Team
    **Threat Model:** STRIDE Analysis completed 2026-01-15

servers:
  - url: https://api.example.com/v1

security:
  - OAuth2: [read:customers, write:customers]

paths:
  /customers/{id}:
    get:
      summary: Retrieve customer data
      security:
        - OAuth2: [read:customers]
      parameters:
        - name: id
          in: path
          required: true
          schema:
            type: string
            pattern: '^[0-9a-f]{8}-[0-9a-f]{4}-4[0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12}$'  # UUID v4 only
      responses:
        '200':
          description: Customer data (Confidential)
          headers:
            X-Request-ID:
              description: For audit tracing
              schema:
                type: string
```

**Automated API Discovery:**
Tools like **Kiterunner** or **OWASP Amass** can discover APIs through DNS enumeration and brute-forcing common endpoint patterns, helping identify undocumented "rogue" endpoints.

---

## 5.2 Business Context & Impact Analysis

Not all assets are equal. A vulnerability in the cafeteria menu app does not equal a vulnerability in the payment processing gateway. **Business Impact Analysis (BIA)** establishes this criticality context.

### 5.2.1 Asset Criticality Classification
Classify assets based on their importance to business operations:

| Tier | Classification | Criteria | Recovery Time Objective (RTO) | Example |
|------|---------------|----------|------------------------------|---------|
| 1 | **Mission Critical** | Revenue-generating, regulatory compliance, safety-of-life | < 4 hours | Payment gateway, Trading platform |
| 2 | **Business Critical** | Supports core operations, high reputational impact | < 24 hours | CRM, Email systems |
| 3 | **Business Important** | Productivity tools, limited operational impact | < 72 hours | HR portal, Internal wiki |
| 4 | **Routine** | Non-essential, easily replaced | < 1 week | Marketing blog, Test environments |

### 5.2.2 Data Classification and Handling
Data classification determines encryption requirements, access controls, and retention policies. The **NIST SP 800-60** guidelines provide a federal standard adaptable to industry.

**Classification Levels:**
1.  **Public:** Approved for public disclosure (marketing materials).
2.  **Internal:** Organization use only, but disclosure causes no harm (org charts).
3.  **Confidential:** Unauthorized disclosure causes moderate harm (financial data, customer lists).
4.  **Restricted/Regulated:** Unauthorized disclosure causes severe harm or violates law (PHI under HIPAA, PCI-DSS data, trade secrets).

**Implementation: Automated Data Classification**
Use regular expressions and machine learning to scan repositories and databases for unclassified sensitive data:

```python
import re
import hashlib

class DataClassifier:
    def __init__(self):
        # Patterns for PII/PHI detection
        self.patterns = {
            'SSN': r'\b\d{3}-\d{2}-\d{4}\b',
            'CREDIT_CARD': r'\b(?:\d[ -]*?){13,16}\b',
            'EMAIL': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'API_KEY': r'\b(sk|pk)_(live|test)_[a-zA-Z0-9]{24,}\b',  # Stripe-like keys
            'HIPAA_MEDICAL_RECORD': r'\bMRN:\s*\d{6,10}\b'
        }
    
    def scan_text(self, content, source):
        """
        Scan text content for sensitive data patterns
        Returns classification recommendation
        """
        findings = {}
        total_severity = 0
        
        for data_type, pattern in self.patterns.items():
            matches = re.findall(pattern, content)
            if matches:
                # Hash matches for privacy in logs
                hashed_matches = [hashlib.sha256(m.encode()).hexdigest()[:8] for m in matches]
                findings[data_type] = {
                    'count': len(matches),
                    'examples': hashed_matches[:3],  # Log only hashes, not actual data
                    'severity': self._get_severity(data_type)
                }
                total_severity += len(matches) * self._get_severity(data_type)
        
        # Determine classification based on findings
        if 'HIPAA_MEDICAL_RECORD' in findings or total_severity > 10:
            classification = 'Restricted'
        elif 'CREDIT_CARD' in findings or 'SSN' in findings:
            classification = 'Confidential'
        elif total_severity > 0:
            classification = 'Internal'
        else:
            classification = 'Public'
        
        return {
            'source': source,
            'recommended_classification': classification,
            'findings': findings,
            'requires_encryption': classification in ['Confidential', 'Restricted'],
            'retention_years': 7 if classification == 'Restricted' else 3
        }
    
    def _get_severity(self, data_type):
        severity_map = {
            'HIPAA_MEDICAL_RECORD': 5,
            'CREDIT_CARD': 4,
            'SSN': 4,
            'API_KEY': 3,
            'EMAIL': 1
        }
        return severity_map.get(data_type, 1)

# Usage for scanning code repositories
# classifier = DataClassifier()
# with open('config.py', 'r') as f:
#     result = classifier.scan_text(f.read(), 'config.py')
```

### 5.2.3 Dependency Mapping
Understanding **data flow** is as important as knowing the assets themselves. A database may be Restricted, but if a Business Important app has direct access to it, the app's criticality effectively increases.

**Data Flow Diagrams (DFD):**
Create DFDs showing:
*   Data sources (databases, external APIs)
*   Processes (microservices, functions)
*   Data stores (caches, object storage)
*   Trust boundaries (internet, DMZ, internal network)

---

## 5.3 Threat Modeling Methodologies: STRIDE, PASTA, and LINDDUN

Threat modeling is the practice of identifying and eliminating potential threats **before** code is written. It is the ultimate "shift left" activity—fixing architectural flaws in design is exponentially cheaper than fixing them in production.

### 5.3.1 The STRIDE Framework (Microsoft)
STRIDE categorizes threats by the type of attack. It is the most accessible framework for developers.

| Threat | Property Violated | Description | Example |
|--------|------------------|-------------|---------|
| **S**poofing | Authentication | Pretending to be someone/something else | Stolen credentials, forging JWT tokens |
| **T**ampering | Integrity | Modifying data or code | SQL injection, modifying request parameters |
| **R**epudiation | Non-repudiation | Denying an action took place | Lack of audit logs for financial transactions |
| **I**nformation Disclosure | Confidentiality | Exposing data to unauthorized parties | Verbose error messages, IDOR vulnerabilities |
| **D**enial of Service | Availability | Disrupting service availability | DDoS, resource exhaustion attacks |
| **E**levation of Privilege | Authorization | Gaining unauthorized capabilities | Privilege escalation, broken access control |

**Threat Modeling Process:**
1.  **Decompose:** Create DFDs showing processes, data stores, data flows, and trust boundaries.
2.  **Identify:** For each element, ask STRIDE questions.
3.  **Mitigate:** Design controls to eliminate or reduce threats.
4.  **Validate:** Verify mitigations work and no new threats introduced.

**Tool: pytm (Pythonic Threat Modeling)**
```python
from pytm import TM, Server, Datastore, Dataflow, Boundary

# Create the threat model
tm = TM("Customer Portal Threat Model")
tm.description = "Web application handling PII"
tm.isOrdered = True

# Define boundaries
internet = Boundary("Internet")
dmz = Boundary("DMZ")
internal = Boundary("Internal Network")

# Assets
web_server = Server("Web Server")
web_server.OS = "Linux"
web_server.isHardened = True
web_server.inBoundary = dmz

database = Datastore("PostgreSQL DB")
database.OS = "Linux"
database.isEncryptedAtRest = True
database.inBoundary = internal
database.storesSensitiveData = True

# Data flows
user_login = Dataflow("User", web_server, "Login Credentials")
user_login.protocol = "HTTPS"
user_login.usesEncryption = True

db_query = Dataflow(web_server, database, "SQL Query")
db_query.protocol = "PostgreSQL"
db_query.usesEncryption = True
db_query.usesTLS = True

# Process
tm.process()  # Generates threat list based on STRIDE
```

### 5.3.2 PASTA (Process for Attack Simulation and Threat Analysis)
PASTA is a risk-centric framework with seven stages, ideal for applications with high security requirements (financial, critical infrastructure).

1.  **Define Objectives:** Business impact, compliance requirements.
2.  **Define Technical Scope:** Components, APIs, dependencies.
3.  **Application Decomposition:** DFDs, trust boundaries.
4.  **Threat Analysis:** Real-world threat intelligence (MITRE ATT&CK mapping).
5.  **Vulnerability Detection:** Linking threats to actual vulnerabilities.
6.  **Attack Modeling:** Probability and impact scoring.
7.  **Risk Analysis:** Countermeasures and residual risk.

### 5.3.3 LINDDUN (Privacy Threat Modeling)
While STRIDE covers security, LINDDUN specifically addresses privacy threats required by GDPR/CCPA.

| Threat | Description | Example |
|--------|-------------|---------|
| **Li**nkability | Associating data with user | Combining logs to track user across sessions |
| **Id**entifiability | Identifying anonymous user | Re-identifying anonymized dataset |
| **N**on-repudiation | Denying data sharing consent | User claims they didn't agree to TOS |
| **D**etectability | Determining if data exists | Attackers inferring user existence via error messages |
| **D**isclosure | Unauthorized data access | Data breach exposing user profiles |
| **U**nawareness | Lack of transparency | User unaware of data collection |
| **N**on-compliance | Violating regulations | Retaining data beyond legal requirement |

---

## 5.4 Vulnerability Management: Scanning, Assessment, and Prioritization

Vulnerability management is the continuous cycle of identifying, classifying, remediating, and mitigating software vulnerabilities. It bridges the gap between "potential threats" (STRIDE) and "actual weaknesses" (CVEs).

### 5.4.1 The Vulnerability Landscape
*   **CVE (Common Vulnerabilities and Exposures):** Global dictionary of publicly known security vulnerabilities (e.g., CVE-2021-44228 for Log4j).
*   **CWE (Common Weakness Enumeration):** Categories of software weaknesses (e.g., CWE-89 for SQL Injection).
*   **CVSS (Common Vulnerability Scoring System):** Standardized scoring (0-10) of severity. CVSS v4.0 (released 2023) improves on v3.1 by better incorporating threat intelligence and environmental factors.

**CVSS v4.0 Metrics:**
*   **Base Metrics:** Inherent characteristics (attack vector, complexity, privileges required).
*   **Threat Metrics:** Exploit maturity (Attacked, POC, Unreported).
*   **Environmental Metrics:** Asset-specific impact (confidentiality/integrity requirements).

### 5.4.2 Vulnerability Scanning Types

**1. Network Scanning**
Discovers open ports, services, and missing OS patches.
*   **Tools:** Nessus, OpenVAS, Nmap scripts.

**2. Static Application Security Testing (SAST)**
Analyzes source code without execution. Finds SQLi, XSS, hardcoded secrets.
*   **Tools:** SonarQube, Semgrep, Checkmarx, Bandit (Python).

```yaml
# Example: Semgrep CI rule for detecting SQL injection (SAST)
# .semgrep.yml
rules:
  - id: python-sql-injection
    pattern: |
      cursor.execute($X % ...)
    message: "Possible SQL injection detected. Use parameterized queries."
    languages: [python]
    severity: ERROR
    metadata:
      cwe: "CWE-89: SQL Injection"
      owasp: "A03:2021 - Injection"
```

**3. Dynamic Application Security Testing (DAST)**
Tests running applications by sending malicious payloads.
*   **Tools:** OWASP ZAP, Burp Suite.

**4. Software Composition Analysis (SCA)**
Identifies vulnerable open-source dependencies.
*   **Tools:** Snyk, OWASP Dependency-Check, GitHub Dependabot.

**5. Container Scanning**
Analyzes Docker images for OS-level and application-level vulnerabilities.
*   **Tools:** Trivy, Clair, Grype.

### 5.4.3 Risk-Based Prioritization (Beyond CVSS)
CVSS Base Score alone is insufficient. A "Critical" vulnerability in an isolated test system is less urgent than a "High" vulnerability in a public-facing payment API.

**The Exploit Prediction Scoring System (EPSS):**
EPSS (first.org/epss) uses machine learning to predict the probability that a vulnerability will be exploited in the wild within the next 30 days. It helps prioritize the 2% of vulnerabilities that pose 80% of the risk.

**Prioritization Formula:**
$$ Priority = f(CVSS, EPSS, Asset\_Criticality, Network\_Exposure) $$

**Implementation: Risk-Based Triage Script**
```python
import requests
from datetime import datetime

class RiskBasedPrioritizer:
    def __init__(self):
        self.epss_api = "https://api.first.org/data/v1/epss"
    
    def get_epss_score(self, cve_id):
        """Fetch EPSS probability score for a CVE"""
        try:
            response = requests.get(f"{self.epss_api}?cve={cve_id}")
            data = response.json()
            if data['data']:
                return float(data['data'][0]['epss'])
            return 0.0
        except:
            return 0.0
    
    def calculate_priority(self, vulnerability):
        """
        Calculate priority score (0-100)
        vulnerability dict contains: cve, cvss, asset_tier, exposure
        """
        # Normalize CVSS (0-10 -> 0-40 weight)
        cvss_weight = vulnerability['cvss'] * 4
        
        # EPSS probability (0-1 -> 0-30 weight)
        epss = self.get_epss_score(vulnerability['cve'])
        epss_weight = epss * 30
        
        # Asset criticality (Tier 1=25, Tier 2=15, Tier 3=5, Tier 4=1)
        tier_weights = {'Tier1': 25, 'Tier2': 15, 'Tier3': 5, 'Tier4': 1}
        asset_weight = tier_weights.get(vulnerability['asset_tier'], 1)
        
        # Exposure (Internet=5, Internal=2, Isolated=0)
        exposure_weights = {'Internet': 5, 'Internal': 2, 'Isolated': 0}
        exposure_weight = exposure_weights.get(vulnerability['exposure'], 0)
        
        total_score = cvss_weight + epss_weight + asset_weight + exposure_weight
        
        # Determine SLA
        if total_score >= 80:
            sla = "24 Hours"
            priority = "Critical"
        elif total_score >= 60:
            sla = "72 Hours"
            priority = "High"
        elif total_score >= 40:
            sla = "7 Days"
            priority = "Medium"
        else:
            sla = "30 Days"
            priority = "Low"
        
        return {
            'cve': vulnerability['cve'],
            'total_score': round(total_score, 2),
            'priority': priority,
            'remediation_sla': sla,
            'epss_probability': f"{epss:.2%}",
            'factors': {
                'cvss_contribution': cvss_weight,
                'epss_contribution': epss_weight,
                'asset_contribution': asset_weight,
                'exposure_contribution': exposure_weight
            }
        }

# Example usage
# vuln = {
#     'cve': 'CVE-2021-44228',
#     'cvss': 10.0,
#     'asset_tier': 'Tier1',
#     'exposure': 'Internet'
# }
# prioritizer = RiskBasedPrioritizer()
# print(prioritizer.calculate_priority(vuln))
```

---

## 5.5 Risk Assessment & Treatment: Quantitative vs. Qualitative Approaches

With assets identified, threats modeled, and vulnerabilities cataloged, we must assess the risk they collectively represent and decide how to treat that risk.

### 5.5.1 Qualitative Risk Assessment
Uses categorical scales (High/Medium/Low) based on expert judgment. Fast, suitable for initial screenings.

**Risk Matrix (5x5):**
| Likelihood \ Impact | Negligible | Minor | Moderate | Major | Catastrophic |
|---------------------|------------|-------|----------|-------|--------------|
| **Almost Certain** | Medium | High | Critical | Critical | Critical |
| **Likely** | Medium | Medium | High | Critical | Critical |
| **Possible** | Low | Medium | High | High | Critical |
| **Unlikely** | Low | Low | Medium | High | High |
| **Rare** | Low | Low | Low | Medium | Medium |

### 5.5.2 Quantitative Risk Assessment (FAIR Model)
The **Factor Analysis of Information Risk (FAIR)** model provides a framework for understanding, analyzing, and measuring information risk in financial terms ($).

**FAIR Terminology:**
*   **Loss Event Frequency (LEF):** How often does the threat contact the asset and succeed?
    *   Threat Event Frequency (TEF) × Vulnerability (Threat Capability vs Control Strength)
*   **Loss Magnitude (LM):** How much does it cost when it happens?
    *   Primary Loss (response, replacement) + Secondary Loss (fines, reputation, customer churn).

**Calculation:**
$$ Risk = Loss Event Frequency × Loss Magnitude $$

**Implementation: Simplified FAIR Calculation**
```python
import numpy as np

class FairRiskAnalyzer:
    def __init__(self):
        # Calibrated estimates (using ranges for Monte Carlo simulation)
        self.simulations = 10000
    
    def calculate_lef(self, threat_contact_freq, probability_of_success):
        """
        Loss Event Frequency = Contact Frequency * Probability of Action * Probability of Success
        """
        # Monte Carlo simulation with triangular distributions
        tef = np.random.triangular(
            threat_contact_freq['min'],
            threat_contact_freq['mode'],
            threat_contact_freq['max'],
            self.simulations
        )
        
        vuln = np.random.triangular(
            probability_of_success['min'],
            probability_of_success['mode'],
            probability_of_success['max'],
            self.simulations
        )
        
        return tef * vuln
    
    def calculate_lm(self, primary_loss, secondary_loss):
        """Total Loss Magnitude"""
        pl = np.random.triangular(
            primary_loss['min'],
            primary_loss['mode'],
            primary_loss['max'],
            self.simulations
        )
        
        sl = np.random.triangular(
            secondary_loss['min'],
            secondary_loss['mode'],
            secondary_loss['max'],
            self.simulations
        )
        
        return pl + sl
    
    def analyze_risk(self, scenario):
        """
        scenario = {
            'tef': {'min': 1, 'mode': 5, 'max': 20},  # times per year
            'vuln': {'min': 0.1, 'mode': 0.3, 'max': 0.8},  # probability
            'primary_loss': {'min': 10000, 'mode': 50000, 'max': 200000},
            'secondary_loss': {'min': 0, 'mode': 100000, 'max': 500000}  # regulatory fines
        }
        """
        lef = self.calculate_lef(scenario['tef'], scenario['vuln'])
        lm = self.calculate_lm(scenario['primary_loss'], scenario['secondary_loss'])
        
        annual_loss_exposure = lef * lm
        
        return {
            'ale_mean': np.mean(annual_loss_exposure),
            'ale_90th': np.percentile(annual_loss_exposure, 90),
            'ale_max': np.max(annual_loss_exposure),
            'interpretation': self._interpret_ale(np.mean(annual_loss_exposure))
        }
    
    def _interpret_ale(self, ale):
        if ale > 1000000:
            return "Critical: Risk treatment required immediately"
        elif ale > 100000:
            return "High: Senior management attention required"
        elif ale > 10000:
            return "Medium: Risk owner monitoring required"
        else:
            return "Low: Acceptable with routine monitoring"

# Example: Data breach risk for customer database
# fair = FairRiskAnalyzer()
# result = fair.analyze_risk({
#     'tef': {'min': 0.5, 'mode': 2, 'max': 5},  # Attack attempts per year
#     'vuln': {'min': 0.05, 'mode': 0.2, 'max': 0.5},  # 20% chance of success (patched systems)
#     'primary_loss': {'min': 50000, 'mode': 150000, 'max': 400000},  # Forensics, notification
#     'secondary_loss': {'min': 100000, 'mode': 500000, 'max': 2000000}  # GDPR fines
# })
# print(f"Annual Loss Exposure (Average): ${result['ale_mean']:,.2f}")
```

### 5.5.3 Risk Treatment Options
Once risk is quantified, we apply the four treatment strategies from Chapter 1:

1.  **Mitigate:** Implement controls to reduce likelihood or impact (firewalls, encryption, training). Most common for high risks.
2.  **Transfer:** Cyber insurance, outsourcing, cloud provider shared responsibility. Does not eliminate risk but shifts financial impact.
3.  **Accept:** Acknowledge and monitor. Appropriate for low risks or when mitigation cost exceeds potential loss. Requires documented approval.
4.  **Avoid:** Discontinue the activity or system. Extreme but effective for unacceptable risks (e.g., shutting down a vulnerable legacy system rather than securing it).

**Risk Register Maintenance:**
A living document tracking:
*   Risk ID, Description, Owner
*   Inherent Risk (before controls)
*   Control Effectiveness
*   Residual Risk (after controls)
*   Treatment Strategy
*   Target Date for Remediation

---

### Chapter Summary

In this chapter, we executed the **IDENTIFY** function with rigor and precision. We implemented **automated asset discovery** to eliminate shadow IT and map our ephemeral cloud resources, recognizing that comprehensive visibility is the prerequisite for all security operations. We established **business context** through data classification and criticality tiers, ensuring that security efforts align with business value and regulatory requirements. We applied **threat modeling** methodologies—STRIDE for security design, PASTA for risk-centric analysis, and LINDDUN for privacy—to anticipate attacks before code is committed. We implemented a **vulnerability management program** that moves beyond CVSS scores to risk-based prioritization using EPSS and asset criticality, ensuring we patch what matters most first. Finally, we quantified risk using the **FAIR model**, translating technical vulnerabilities into financial terms that drive executive decision-making.

Identification gives us the map and the intelligence, but intelligence without action is mere observation. Having cataloged our assets, modeled our threats, and assessed our risks, we must now implement the safeguards—the technical, administrative, and physical controls that constitute the **PROTECT** function. We transition from knowing what to defend to actively building the walls, gates, and guardians that enforce our security policies.

**Next Up: Chapter 6: PROTECT (PR) – Safeguards & Implementation**